# 𝕆𝕦𝕥 𝕠𝕗 𝕕𝕚𝕤𝕥𝕣𝕚𝕓𝕦𝕥𝕚𝕠𝕟

Source:
* https://github.com/bentrevett/pytorch-seq2seq/blob/master/6%20-%20Attention%20is%20All%20You%20Need.ipynb

In [ ]:
import os


import torch
import torch.nn
import torch.optim as optim
from torch.autograd import Variable

import torchtext
from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator






import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time


SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128



# Prepare Data

In [ ]:
spacy_vae_features = spacy.load('de_core_news_sm')
spacy_i_dont_know = spacy.load('en_core_web_sm')

def tokenize_vae_features(text):
    """
    Tokenizes vae autoencoders from a numpy array
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_i_dont_know(text):
    """
    Class labels???
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]


SRC = Field(tokenize = tokenize_vae_features, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True)

TRG = Field(tokenize = tokenize_i_dont_know, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True)



# Dataset

In [ ]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)


train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
     batch_size = BATCH_SIZE,
     device = device)



# Model
* https://pytorch.org/text/main/tutorials/t5_demo.html#generate-sentiment-classifications

In [ ]:
batch = next(iter(imdb_dataloader))
input_text = batch["text"]
target = batch["label"]
beam_size = 1

model_input = transform(input_text)
model_output = sequence_generator.generate(model_input, eos_idx=eos_idx, num_beams=beam_size)
output_text = transform.decode(model_output.tolist())

for i in range(imdb_batch_size):
    print(f"Example {i+1}:\n")
    print(f"input_text: {input_text[i]}\n")
    print(f"prediction: {output_text[i]}\n")
    print(f"target: {target[i]}\n\n")

In [1]:
# Example - Pretrained large xlmr encoder attached to un-initialized classification head

import torch, torchtext
from torchtext.models import RobertaClassificationHead
from torchtext.functional import to_tensor
xlmr_large = torchtext.models.XLMR_LARGE_ENCODER
classifier_head = torchtext.models.RobertaClassificationHead(num_classes=2, input_dim = 1024)
model = xlmr_large.get_model(head=classifier_head)
transform = xlmr_large.transform()
input_batch = ["Hello world", "How are you!"]
model_input = to_tensor(transform(input_batch), padding_value=1)
output = model(model_input)
output.shape



ModuleNotFoundError: No module named 'torchtext'

In [2]:
from torchtext.models import RobertaEncoderConf, RobertaBundle, RobertaClassificationHead
model_weights_path = "https://download.pytorch.org/models/text/xlmr.base.encoder.pt"
encoder_conf = RobertaEncoderConf(vocab_size=250002)
classifier_head = RobertaClassificationHead(num_classes=18, input_dim=768)
model = RobertaBundle.build_model(encoder_conf=encoder_conf, head=classifier_head, checkpoint=model_weights_path)

ModuleNotFoundError: No module named 'torchtext'

In [ ]:

class OODModel(torch.nn.Module):
    # =============================================================================
    # use this
    # encoder backbone (shufflenet_v2_x1_0) + projection head (MLP)
    # output size of MLP: 128 
    # this feature vector is then used for the SupConLoss

    # Replace model head:
    # https://discuss.pytorch.org/t/how-to-replace-a-models-head/109002/2
    # shuffle net: (fc): Linear(in_features=1024, out_features=1000, bias=True)

    # this function is used to train a DecentBlock with the SupConLoss
    # =============================================================================
    
    def __init__(self, out_classes=20):
        super(OODModel, self).__init__()
        
        self.out_classes = out_classes
        
        # encoder
        try:
            shufflenet = shufflenet_v2_x1_0(weights=ShuffleNet_V2_X1_0_Weights.IMAGENET1K_V1)
        except: 
            shufflenet = shufflenet_v2_x1_0(pretrained=True)
                    
        # size of encoder output
        encoder_out = shufflenet.fc.in_features
            
        # placeholder identity operator 
        shufflenet.fc = torch.nn.Identity()
                
        # encoder without fully connected classification head (linear layer)
        self.encoder = shufflenet
        
        # maybe we need concatenation or view change here ... 1x1 conv??
        
        # MLP head
        self.mlp_head = torch.nn.Sequential(
            torch.nn.Linear(encoder_out, encoder_out),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(encoder_out, 128)
        )
        
        self.classification = torch.nn.Sequential(
            torch.nn.Linear(128, self.out_classes)
        )
        

    def forward(self, x):
        x = self.encoder(x)
        x = self.mlp_head(x)
        block_output = torch.nn.functional.normalize(x, dim=1)
        
        
        
        return block_output